기준금리 크롤링

In [1]:
import requests
import datetime
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

base_url= "https://www.bok.or.kr/portal/singl/baseRate/list.do?dataSeCd=01&menuNo=200643"
response = requests.get(base_url, 'html.parser')
soup= BeautifulSoup(response.text, 'lxml')

In [2]:
dict_list= []
# 53
for i in range(53):
    dict_list.append({
        'DATE': pd.to_datetime(datetime.datetime.strptime(soup.find_all('td')[3*i].text + soup.find_all('td')[3*i+1].text, '%Y%m월 %d일')),
        'RATE': float(soup.find_all('td')[3*i+2].text)
    })

In [3]:
pd_result= pd.DataFrame(dict_list)

# date range 일별 추가
idx= pd.date_range(pd_result.DATE.min(), pd_result.DATE.max())
idx= pd.Series(idx)

In [4]:
# 없는 행에 Nan을 넣어 병합한 후 Nan값을 이전 값으로 메꾸기
baserate= pd.concat([pd.DataFrame({'DATE': idx[~idx.isin(pd_result.DATE)], 'RATE': np.nan}),pd_result]).sort_values('DATE').reset_index(drop=True).ffill(axis=0)

In [5]:
baserate

,DATE,RATE
0,1999-05-06,4.75
1,1999-05-07,4.75
2,1999-05-08,4.75
3,1999-05-09,4.75
4,1999-05-10,4.75
...,...,...
8556,2022-10-08,2.50
8557,2022-10-09,2.50
8558,2022-10-10,2.50
8559,2022-10-11,2.50


In [6]:
baserate.to_csv('./baserate.csv')

콜금리

In [7]:
call_url= "https://finance.naver.com/marketindex/interestDailyQuote.nhn?marketindexCd=IRR_CALL&page={}"
page_num= 622 # 2005년 5월~

In [8]:
call_list= []

for i in range(1,page_num+1):
    call_list.append(BeautifulSoup(requests.get(call_url.format(i), 'html.parser').text,'lxml').find_all('td'))

In [9]:
result_list= []

for list in call_list:
    
    for i in range(len(list)//4):
        result_list.append({
            'DATE': list[4*i].text.strip(),
            'RATE': float(list[4*i+1].text)
        })

In [10]:
result_list

[{'DATE': '2022.11.17', 'RATE': 3.02},
 {'DATE': '2022.11.16', 'RATE': 3.03},
 {'DATE': '2022.11.15', 'RATE': 3.03},
 {'DATE': '2022.11.14', 'RATE': 3.07},
 {'DATE': '2022.11.11', 'RATE': 3.07},
 {'DATE': '2022.11.10', 'RATE': 3.09},
 {'DATE': '2022.11.09', 'RATE': 2.98},
 {'DATE': '2022.11.08', 'RATE': 2.99},
 {'DATE': '2022.11.07', 'RATE': 2.97},
 {'DATE': '2022.11.04', 'RATE': 2.99},
 {'DATE': '2022.11.03', 'RATE': 3.04},
 {'DATE': '2022.11.02', 'RATE': 3.03},
 {'DATE': '2022.11.01', 'RATE': 3.08},
 {'DATE': '2022.10.31', 'RATE': 3.19},
 {'DATE': '2022.10.28', 'RATE': 3.04},
 {'DATE': '2022.10.27', 'RATE': 3.07},
 {'DATE': '2022.10.26', 'RATE': 3.12},
 {'DATE': '2022.10.25', 'RATE': 3.14},
 {'DATE': '2022.10.24', 'RATE': 3.13},
 {'DATE': '2022.10.21', 'RATE': 3.1},
 {'DATE': '2022.10.20', 'RATE': 3.13},
 {'DATE': '2022.10.19', 'RATE': 3.07},
 {'DATE': '2022.10.18', 'RATE': 3.08},
 {'DATE': '2022.10.17', 'RATE': 3.05},
 {'DATE': '2022.10.14', 'RATE': 3.07},
 {'DATE': '2022.10.13', 'R

In [11]:
pd_result2= pd.DataFrame(result_list)
pd_result2.DATE= pd_result2.DATE.astype('datetime64')

In [12]:
# 원하는 전체 date range
idx2= pd.date_range(pd_result2.DATE.min(), pd_result2.DATE.max())
idx2= pd.Series(idx2)

In [13]:
# 없는 행에 Nan을 넣어 병합한 후 Nan값을 이전 값으로 메꾸기
callrate= pd.concat([pd.DataFrame({'DATE': idx2[~idx2.isin(pd_result2.DATE)], 'RATE': np.nan}),pd_result2]).sort_values('DATE').reset_index(drop=True).ffill(axis=0)

In [14]:
callrate

,DATE,RATE
0,2005-05-02,3.29
1,2005-05-03,3.30
2,2005-05-04,3.21
3,2005-05-05,3.21
4,2005-05-06,3.27
...,...,...
6404,2022-11-13,3.07
6405,2022-11-14,3.07
6406,2022-11-15,3.03
6407,2022-11-16,3.03


In [15]:
callrate.to_csv('./callrate.csv')